In [1]:
using Revise

using Test
import TensorCrossInterpolation as TCI
import Random

In [2]:
using Test
import TensorCrossInterpolation as TCI
import TensorCrossInterpolation: rank, linkdims, TensorCI2, updatepivots!, addglobalpivots1sitesweep!, MultiIndex, evaluate, crossinterpolate2, pivoterror, tensortrain
import Random
import QuanticsGrids as QD

In [3]:
import TensorCrossInterpolation as TCI
using TensorCrossInterpolation
using Random
using Test

In [17]:
pivotsearch = :full
seed = 123
strictlynested = false

Random.seed!(seed)

R = 20
abstol = 1e-4
δ = 10.0 / 2^R # Peaks are wider than 1/2^R.
grid = QD.DiscretizedGrid{1}(R, (0.0,), (1.0,))

rindex = [rand(1:2, R) for _ in 1:100]

f(bitlist) = any(bitlist .> 2) ? 0.0 : fx(QD.quantics_to_origcoord(grid, bitlist)[1])
rpoint = Float64[QD.quantics_to_origcoord(grid, r)[1] for r in rindex]

function fx(x)
    # res = 0.5 * (exp(-10 * x) + exp(-x)) For this function, two-site update finds all the pivots!
    res = exp(-10 * x)
    for r in rpoint
        res += abs(x - r) < δ ? 2 * abstol : 0.0
    end
    res
end

localdims = fill(2, R)
firstpivot = ones(Int, R)
tci, ranks, errors = crossinterpolate2(
    Float64,
    f,
    localdims,
    [firstpivot];
    tolerance=abstol,
    maxbonddim=1000,
    maxiter=20,
    loginterval=1,
    verbosity=1,
    normalizeerror=false,
    pivotsearch=pivotsearch,
    nsearchglobalpivot=0
)

iteration = 1, rank = 1, error= 1.1102230246251565e-16, maxsamplevalue= 1.0, nglobalpivot=0
iteration = 2, rank = 1, error= 1.1102230246251565e-16, maxsamplevalue= 1.0, nglobalpivot=0
iteration = 3, rank = 1, error= 1.1102230246251565e-16, maxsamplevalue= 1.0, nglobalpivot=0


(TensorCI2{Float64} with rank 1, [1, 1, 1], [1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16])

In [18]:
#==
@show sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex]) .> abstol) == 0

rindex_test = [rand(1:2, R) for _ in 1:100]

@show sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex_test]) .> abstol) == 0
==#

In [19]:
TCI.addglobalpivots2sitesweep!(tci, f, rindex; fillsitetensors=false, verbosity=4)

    Computing Pi (2 x 202) at bond 1: 0.005940333000000001 sec, LU: 1.8500000000000002e-5 sec
    Computing Pi (4 x 202) at bond 2: 0.009684042 sec, LU: 2.2791000000000003e-5 sec
    Computing Pi (8 x 202) at bond 3: 0.018295375000000003 sec, LU: 3.55e-5 sec
    Computing Pi (16 x 202) at bond 4: 0.038490125 sec, LU: 8.1333e-5 sec
    Computing Pi (32 x 202) at bond 5: 0.072866083 sec, LU: 0.000218292 sec
    Computing Pi (60 x 202) at bond 6: 0.14582166700000002 sec, LU: 0.000608666 sec
    Computing Pi (100 x 202) at bond 7: 0.23077970800000003 sec, LU: 0.001445667 sec
    Computing Pi (134 x 196) at bond 8: 0.304207875 sec, LU: 0.002841625 sec
    Computing Pi (164 x 194) at bond 9: 0.5454979590000001 sec, LU: 0.00364175 sec
    Computing Pi (182 x 176) at bond 10: 0.378462625 sec, LU: 0.0034240420000000004 sec
    Computing Pi (188 x 162) at bond 11: 0.35548683400000003 sec, LU: 0.002676583 sec
    Computing Pi (196 x 138) at bond 12: 0.303163625 sec, LU: 0.0022522500000000003 sec


In [20]:
for b in 1:length(tci)-1
    @show length(TCI.Iset(tci, b+1)), length(TCI.Jset(tci, b))
end

(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (2, 746)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (4, 745)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (8, 743)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (16, 738)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (31, 720)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (52, 704)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (71, 671)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (87, 600)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (103, 521)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (110, 440)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (122, 318)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (126, 204)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (139, 120)
(length(TCI.Iset(tci, b + 1)), length(TCI.Jset(tci, b))) = (172, 64)
(length(TCI.Iset(tci, b + 1)), l

In [21]:
TCI.linkdims(tci)

19-element Vector{Int64}:
   2
   4
   8
  16
  31
  52
  71
  87
 103
 110
 122
 126
 120
  64
  32
  16
   8
   4
   2

In [22]:
TCI.fillsitetensors!(tci, f)

In [23]:
length(tci.globalpivots)

746

In [25]:
tt = TCI.TensorTrain(tci)
@show TCI.linkdims(tt)
TCI.compress!(tt, :SVD; tolerance=1e-14)
@show TCI.linkdims(tt)

TCI.linkdims(tt) = [2, 4, 8, 16, 31, 51, 68, 82, 91, 93, 90, 85, 79, 60, 32, 16, 8, 4, 2]
TCI.linkdims(tt) = [2, 4, 8, 16, 31, 51, 68, 82, 91, 93, 90, 85, 79, 60, 32, 16, 8, 4, 2]


19-element Vector{Int64}:
  2
  4
  8
 16
 31
 51
 68
 82
 91
 93
 90
 85
 79
 60
 32
 16
  8
  4
  2

In [26]:
@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex]) .> abstol) == 0

rindex_test = [rand(1:2, R) for _ in 1:100]

@test sum(abs.([TCI.evaluate(tci, r) - f(r) for r in rindex_test]) .> abstol) == 0

Test Passed